# Detecting of depression from social network data

The purpose of this project is to determine how Twitter, a well-known social media site, may be used to monitor users for early signs of depression. The study is inspired by following articles: [Early Detection of Depression: Social Network Analysis and Random Forest Techniques](https://www.jmir.org/2019/6/e12554/), [Depression detection from social network data using machine learning techniques](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6111060/).

## Imports
Setup all libraries

In [52]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords

import preprocessor as p

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import train_test_split

## Read the data
Use data from [Sentiment140 dataset](https://www.kaggle.com/datasets/kazanova/sentiment140)

In [53]:
N = 10000

labels = ['target', 'id', 'date', 'query', 'name', 'text']

data = pd.read_csv('data_twitter.csv', names=labels, encoding='ISO-8859-1')

sample = data.sample(N, random_state=123)

print(sample.head())

print(sample[sample['target'] == 0].shape)

         target          id                          date     query  \
448282        0  2068921155  Sun Jun 07 14:56:42 PDT 2009  NO_QUERY   
1475261       4  2065871668  Sun Jun 07 09:27:21 PDT 2009  NO_QUERY   
132529        0  1835774749  Mon May 18 06:43:27 PDT 2009  NO_QUERY   
182348        0  1967121891  Fri May 29 19:00:46 PDT 2009  NO_QUERY   
907614        4  1695846172  Mon May 04 07:04:29 PDT 2009  NO_QUERY   

                   name                                               text  
448282    smiley_sophie  my arm still hurts from when i pulled it yeste...  
1475261  ImmaChocoholic  I have so much to do outside! Been looking at ...  
132529       drmomentum  @AbsolutSara Yes, I knew about the clusterfark...  
182348      sweetsheilx  Just woke up and i feel relieved Haha now i ha...  
907614        monmariej  LOVING the hot weather forecast for the rest o...  
(5087, 6)


## Preprocess data

Replace values in 'target' column

In [54]:
sample['target'] = sample['target'].map({0: 1, 4: 0})
sample = sample.drop(['name', 'id', 'date', 'query'], axis=1)
X = sample.drop(['target'], axis=1)
y = sample['target']

In [55]:
print(sample[sample['target'] == 1].shape)
print(sample.info())

(5087, 2)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 448282 to 113970
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   target  10000 non-null  int64 
 1   text    10000 non-null  object
dtypes: int64(1), object(1)
memory usage: 234.4+ KB
None


### Get rid of contraction, punctuation, and stop words

In [56]:
cont_list = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you're": "you are",
  "you've": "you have"
}

cont_re = re.compile('(%s)' % '|'.join(cont_list.keys()))

def expand_contractions(text):
    def replace(match):
        return cont_list[match.group(0)]
    return cont_re.sub(replace, text)

In [57]:
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def clean_messages(messages):
    stop_words = set(stopwords.words('english'))
    nltk.download('punkt')
    cleaned_messages = []
    for msg in messages:
        msg = str(msg).lower()
        msg = BAD_SYMBOLS_RE.sub(' ', msg)
        msg = p.clean(msg)

        msg = expand_contractions(msg)

        msg = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", msg).split())

        word_tokens = nltk.tokenize.word_tokenize(msg)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        msg = ' '.join(filtered_sentence)
        cleaned_messages.append(msg)

    return cleaned_messages

In [58]:
cleaned_msgs = clean_messages([x for x in X['text']])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gee8w\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Tokenization

In [59]:
NUM_WORDS = 10000
tokenizer= Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(cleaned_msgs)

In [60]:
word_vector = tokenizer.texts_to_sequences(cleaned_msgs)
word_index = tokenizer.word_index
vocab_size = len(word_index)
vocab_size

17770

In [61]:
MAX_SEQ_LENGTH = 140
input_tensor = pad_sequences(word_vector, maxlen=MAX_SEQ_LENGTH)
input_tensor.shape

(10000, 140)

## Logistic Regression

In [62]:


def print_clf_metrics( y_actual, y_pred ):
  print(f'Testing accuracy  = {metrics.accuracy_score(y_actual, y_pred)}')
  print(f'Testing precision = {metrics.precision_score(y_actual, y_pred)}')
  print(f'Testing recall    = {metrics.recall_score(y_actual, y_pred)}')
  print(f'Testing F1-score  = {metrics.f1_score(y_actual, y_pred)}')

parameters = [{'penalty' : ['l2'],
               'C' : np.logspace(-3, 3, 7),
               'solver' : ['newton-cg', 'lbfgs', 'liblinear']},
              {'penalty' : ['l1'],
               'C' : np.logspace(-3, 3, 7),
               'solver' : ['liblinear']}]

In [63]:
# Define Grid Search for Logistic Regression with parameters from above
gs_clf = GridSearchCV(LogisticRegression(max_iter=1000), parameters)

X_train, X_test, y_train, y_test = train_test_split(input_tensor, y, random_state=54, stratify=y)

gs_clf.fit(X_train, y_train)

print("Tuned Hyperparameters :", gs_clf.best_params_)
print("Accuracy :", gs_clf.best_score_)

C:\Users\gee8w\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_linesearch.py:416: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
C:\Users\gee8w\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\gee8w\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
C:\Users\gee8w\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_linesearch.py:416: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
C:\Users\gee8w\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_linesearch.py:306: LineSearchWarning: The line search algorithm did

Tuned Hyperparameters : {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Accuracy : 0.5144


In [64]:
lr_clf = gs_clf.best_estimator_
y_pred = lr_clf.predict(X_test)
print_clf_metrics(y_test, y_pred)

Testing accuracy  = 0.5444
Testing precision = 0.535887749595251
Testing recall    = 0.7806603773584906
Testing F1-score  = 0.6355200000000001


## Naive Bayes

In [65]:
from sklearn.naive_bayes import GaussianNB

# Declare GNB
gauss_nb = GaussianNB()

gauss_nb.fit(X_train, y_train)
y_pred = gauss_nb.predict(X_test)

print_clf_metrics(y_test, y_pred)

Testing accuracy  = 0.5116
Testing precision = 0.510353227771011
Testing recall    = 0.9882075471698113
Testing F1-score  = 0.6730923694779116


##

## LTSM Model (in progress)